# Data Loading and PreProcessing

In [236]:
import sys
import cv2
import os
import glob
import math
from skimage.io import imread
from skimage.transform import resize
from skimage import color
from google.colab import files
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from keras.applications.vgg19 import VGG19
from tensorflow import keras
from torchvision.transforms import InterpolationMode
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras import backend as K
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [3]:
train = files.upload() #upload Train.csv

Saving Train.csv to Train.csv


In [262]:
sf= pd.read_csv('Train.csv')

In [5]:
sf.shape

(298, 2)

In [6]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Frame_ID  298 non-null    object
 1   Emotion   298 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB


In [7]:
train_video = files.upload() #upload Train Video

Saving Train Tom and jerry.mp4 to Train Tom and jerry.mp4


In [8]:
test_video = files.upload() #upload Test video

Saving Test Tom and Jerry.mp4 to Test Tom and Jerry.mp4


In [17]:
sf['Emotion'].value_counts()

Unknown      116
surprised     68
sad           40
happy         37
angry         37
Name: Emotion, dtype: int64

# Image Extraction

In [252]:
if not os.path.exists('images'):
  os.makedirs('images')
imageFolder = "/content/images"
cap = cv2.VideoCapture('Train Tom and jerry.mp4')
frameRate = cap.get(5)
count = 0
while( cap.isOpened() ):
  frameId = cap.get(1)
  ret, frame = cap.read()
  if(ret != True):
    break
  if(frameId % math.floor(frameRate) == 0 ):
    filename = imageFolder + "/frame" + str(count) + ".jpg"
    cv2.imwrite(filename,frame)
    count = count + 1
        
cap.release()
print(count,"Done!")

Done!


In [ ]:
if not os.path.exists('Testimages'):
  os.makedirs('Testimages')
cap = cv2.VideoCapture('/content/Test Tom and Jerry.mp4')
imageFolder = "/content/Testimages"
frameRate = cap.get(5)
count = 0
while( cap.isOpened() ):
  frameId = cap.get(1)
  ret, frame = cap.read()
  if(ret != True):
    break
  if(frameId % math.floor(frameRate) == 0 ):
    filename = imageFolder + "/test" + str(count) + ".jpg"
    cv2.imwrite(filename,frame)
    count = count + 1     
cap.release()
print(count,"Done!")

# Image Transformation

In [260]:
tg_dict = {"angry":0, "happy": 1, "sad": 2,"surprised":3, "Unknown":4}
def label_encode(x):
    return tg_dict[x]

In [263]:
sf['Emotion'] = sf['Emotion'].apply(label_encode)

frame_ID = sf['Frame_ID'].tolist()
emotion = sf['Emotion'].tolist()

In [264]:
features=[]
labels=[]
path = '/content/images/'
sharpenKernel = np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])
#sharpenKernel = np.array([[0,-1,0], [-1, 9,-1], [0,-1,0]])
#sharpenKernel = np.array(([[0, -1, 0], [-1, 9, -1], [0, -1, 0]]), np.float32)/9
#sharpenKernel = np.ones((5,5),np.float32)/25
#sharpenKernel = np.ones((5,5), np.uint8)
transformation = transforms.Compose([
                                      transforms.ToPILImage(),
                                      transforms.CenterCrop((500,720)),
                                      transforms.Resize((224,224), interpolation = InterpolationMode.NEAREST),
                                      
                                    ])
for i in range(len(frame_ID)):
  if os.path.isfile(path+str(frame_ID[i])):
    pic = imread(path+str(frame_ID[i]))
    #plt.imshow(pic)
    #sharpimg = cv2.filter2D(pic,-1,kernel=sharpenKernel)
    #plt.imshow(sharpimg)
    imagetensor = transformation(pic)
    #plt.imshow(imagetensor)
    x = image.img_to_array(imagetensor)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x)
    labels.append(emotion[i])
  else:
    print(path+str(frame_ID[i]), 'not present')

In [265]:
npfeatures = np.array(features)
print(npfeatures.shape)
img_dt = np.rollaxis(npfeatures, 1, 0)
print(img_dt.shape)
X = img_dt[0]
print(X.shape)
labels = np.array(emotion)
Y = to_categorical(emotion,5)
print(Y.shape)

(298, 1, 224, 224, 3)
(1, 298, 224, 224, 3)
(298, 224, 224, 3)
(298, 5)


# Model Build and Train

In [55]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [266]:
IMAGE_SIZE=[224,224]
pretrained_model = MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
pretrained_model.trainable = False

In [237]:
#vgg = VGG19(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE+[3])
#for layer in vgg.layers:
#  layer.trainable = False

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
#vgg.summary()

In [239]:
#xx = Flatten()(vgg.output)
#pre = Dense(5,activation='softmax')(xx)

In [240]:
#model = Model(inputs=vgg.input,outputs=pre)

In [ ]:
#model.summary()

In [267]:
model = Sequential([
        pretrained_model,
        GlobalAveragePooling2D(),
        Dense(60, activation='relu'),
        Dense(30, activation='relu'),
        Dense(5, activation='softmax')
    ])
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, 
                                   verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [281]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=[f1_m])    

In [282]:
model.fit(X, Y, batch_size=32, epochs=50, validation_split=.1,callbacks=[es])

Epoch 1/50
9/9 [==============================] - 12s 1s/step - loss: 0.2138 - f1_m: 0.9402 - val_loss: 1.0344 - val_f1_m: 0.7143
Epoch 2/50
9/9 [==============================] - 10s 1s/step - loss: 0.2289 - f1_m: 0.9216 - val_loss: 1.5098 - val_f1_m: 0.6207
Epoch 3/50
9/9 [==============================] - 10s 1s/step - loss: 0.2096 - f1_m: 0.9522 - val_loss: 0.7702 - val_f1_m: 0.7586
Epoch 4/50
9/9 [==============================] - 10s 1s/step - loss: 0.1196 - f1_m: 0.9826 - val_loss: 0.7616 - val_f1_m: 0.7586
Epoch 5/50
9/9 [==============================] - 10s 1s/step - loss: 0.0928 - f1_m: 0.9814 - val_loss: 1.0994 - val_f1_m: 0.6552
Epoch 6/50
9/9 [==============================] - 10s 1s/step - loss: 0.0828 - f1_m: 0.9916 - val_loss: 0.8118 - val_f1_m: 0.7458
Epoch 7/50
9/9 [==============================] - 10s 1s/step - loss: 0.0651 - f1_m: 0.9940 - val_loss: 0.7813 - val_f1_m: 0.7458
Epoch 8/50
9/9 [==============================] - 10s 1s/step - loss: 0.0533 - f1_m: 0.998

# Predictions

In [59]:
test = files.upload() #upload Test.csv

Saving Test.csv to Test.csv


In [270]:
tf= pd.read_csv('Test.csv')
tf.head()

,Frame_ID
0,test0.jpg
1,test1.jpg
2,test2.jpg
3,test3.jpg
4,test4.jpg


In [271]:
tf.shape

(186, 1)

In [272]:
images_test = tf['Frame_ID'].tolist()
test_features=[]
path_test = '/content/Testimages/'
sharpenKernel = np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])
transformation = transforms.Compose([
                                      transforms.ToPILImage(),
                                      transforms.CenterCrop((500,720)),
                                      transforms.Resize((224,224), interpolation=InterpolationMode.NEAREST),
                                      
                                    ])
for i in range(len(images_test)):
  if os.path.isfile(path_test+str(images_test[i])):
    pic = imread(path_test+str(images_test[i]))
    #sharpimg = cv2.filter2D(pic,-1,kernel=sharpenKernel)
    imagetensor = transformation(pic)
    x = image.img_to_array(imagetensor)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features.append(x)
  else:
    print(path_test+str(images_test[i]), 'not present')

In [273]:
test_features = np.array(test_features)
print(test_features.shape)
test_features = np.rollaxis(test_features, 1, 0)
print(test_features.shape)
X_test = test_features[0]
print(X_test.shape)

(186, 1, 224, 224, 3)
(1, 186, 224, 224, 3)
(186, 224, 224, 3)


In [283]:
preds = model.predict(X_test)

In [284]:
predictions=[]
for i in preds:
    predictions.append(np.argmax(i))

In [285]:
tf['Emotion'] = predictions

In [286]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

tf['Emotion'] = tf['Emotion'].apply(inverse_encode)

In [287]:
tf.head()

,Frame_ID,Emotion
0,test0.jpg,angry
1,test1.jpg,angry
2,test2.jpg,happy
3,test3.jpg,happy
4,test4.jpg,angry


In [288]:
from google.colab import files
tf.to_csv('TomSubmission.csv',header=True,index = None)
files.download('TomSubmission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>